<a href="https://colab.research.google.com/github/annasugian/Sugian_ML_course/blob/main/ML_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Базовый уровень: Генерация текста с использованием LSTM
0. Сбор данных: используйте готовый или соберите свой корпус в формате plain text для генерации текстов

1. Генерация текста на основе небольшого датасета

- Предварительный анализ: чистка текста
- Обучение модели. Используйте образец из туториала по RNNи
- Генерация текста. Используйте образец из туториала по RNN
- Сгенерируйте несколько текстов с помощью созданной модели

## Подготовка датасета: Чистка текста
Текст - 3 романа из серии Джорджа Мартина "Песнь льда и пламени". Сложно было понять, нужно ли оставлять какие-то знаки препинания, напр. вопросительные, или кавычки, ведь это важно для художественной литературы. Но я решила все же оставить только точки.

In [1]:
import re

output_file = 'Song_of_Ice_and_Fire.txt'
input_files = ['1 - A Game of Thrones.txt', '2 - A Clash of Kings.txt', '3 - A Storm of Swords.txt']

with open(output_file, 'w', encoding='ISO-8859-1') as outfile:
    for filename in input_files:
        with open(filename, 'r', encoding='ISO-8859-1') as infile:
            outfile.write(infile.read())


with open('Song_of_Ice_and_Fire.txt', 'r', encoding='ISO-8859-1') as f:
    text = f.read()

pattern = r'\s*Page\s*\d+\s*|[^\w\s.]|\n(?=\w)'
cleanedtext = re.sub(pattern, '', text.lower()).strip()

with open('Clean_SoIaF.txt', 'w', encoding='utf-8') as f:
    f.write(cleanedtext)

with open('Clean_SoIaF.txt', 'r', encoding='utf-8') as f:
    data = f.read()

data = data.split('.')
data[:20]

['we have a long ride before us gared pointed out',
 ' eight days maybe nine',
 ' and night is falling',
 ' ser waymar royce glanced at the sky with disinterest',
 ' it does that every day about this time',
 ' are you unmanned by the dark gared will could see the tightness around gareds mouth the barely sup pressed anger in his eyes under the thick black hood of his cloak',
 ' gared had spent forty years in the nights watch man and boy and he was not accustomed to being made light of',
 ' yet it was more than that',
 ' under the wounded pride will could sense something else in the older man',
 ' you could taste it a nervous tension that came perilous close to fear',
 ' will shared his unease',
 ' he had been four years on the wall',
 ' the first time he had been sent beyond all the old stories had come rushing back and his bowels had turned to water',
 ' he had laughed about it afterward',
 ' he was a veteran of a hundred rangings by now and the endless dark wilderness that the southro

Данные пришлось сильно убавить, так как колаб не справлялся

In [2]:
data = data[:10000]

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

In [4]:
# Инициализируем токенизатор
tokenizer = Tokenizer()

# Обучаем токенизатор на заголовках
tokenizer.fit_on_texts(data)

# Преобразуем заголовки в последовательности чисел
sequences = tokenizer.texts_to_sequences(data)

# Создаем входные и выходные данные
X = []
y = []
for seq in sequences:
    for i in range(1, len(seq)):
        X.append(seq[:i])
        y.append(seq[i])

### Вопрос: Какие значения попадают в y, а что хранится в X'ах?

В Х хранятся входные данные. Это последовательность нарастающей длины, "история" токенов. Х подается на вход нейросети как контекст.

В у хранятся целевые значения. Это следующий токен после текущей последовательности из Х. у это то, что модель должна предсказать на каждом шаге

In [5]:
X[:10], y[:10]

([[81],
  [81, 34],
  [81, 34, 3],
  [81, 34, 3, 101],
  [81, 34, 3, 101, 272],
  [81, 34, 3, 101, 272, 99],
  [81, 34, 3, 101, 272, 99, 151],
  [81, 34, 3, 101, 272, 99, 151, 537],
  [81, 34, 3, 101, 272, 99, 151, 537, 823],
  [951]],
 [34, 3, 101, 272, 99, 151, 537, 823, 55, 350])

In [6]:
# Преобразуем списки в массивы numpy
X = np.asarray(X, dtype="object")
y = np.array(y)

# Дополняем последовательности до одинаковой длины
X = pad_sequences(X)

# Преобразуем y в one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=len(tokenizer.word_index) + 1)

## Создание и обучение модели

In [7]:
# Создаем модель
model = Sequential()

# Добавляем слой Embedding
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X.shape[1]))

# Добавляем слой LSTM
model.add(LSTM(150, return_sequences=False))

# Добавляем полносвязный слой
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))

# Компилируем модель
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Выводим информацию о модели
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Обучаем модель
history = model.fit(X, y, epochs=40, batch_size=64, validation_split=0.2)

Epoch 1/40
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 23s 14ms/step - accuracy: 0.0568 - loss: 6.9139 - val_accuracy: 0.0866 - val_loss: 6.5235
Epoch 2/40
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 37s 13ms/step - accuracy: 0.0992 - loss: 5.9696 - val_accuracy: 0.1059 - val_loss: 6.2882
Epoch 3/40
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - accuracy: 0.1247 - loss: 5.5261 - val_accuracy: 0.1182 - val_loss: 6.2296
Epoch 4/40
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - accuracy: 0.1422 - loss: 5.1808 - val_accuracy: 0.1238 - val_loss: 6.2386
Epoch 5/40
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 18s 13ms/step - accuracy: 0.1531 - loss: 4.9392 - val_accuracy: 0.1259 - val_loss: 6.2908
Epoch 6/40
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 17s 13ms/step - accuracy: 0.1680 - loss: 4.6966 - val_accuracy: 0.1264 - val_loss: 6.3863
Epoch 7/40
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - accuracy: 0.1814 - loss: 4.4759 - val_accuracy: 0.1282 - val_loss: 6.4820
Epoch 8/40
1364/1364 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - accuracy: 0.1937 -

In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 105, 100)            │         819,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 150)                 │         150,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 8192)                │       1,236,992 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,620,378 (25.25 MB)

 Trainable params: 2,206,792 (8.42 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,413,586 (16.84 MB)

### Посмотрим на то, как модель генерирует тексты

In [11]:
# Функция для генерации текста
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Генерируем новый текст
generated_text = generate_text("Winter", 30, X.shape[1])
print(generated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━

### Должно было быть скорее так:
*Winter is coming <s>from us</s> for him. Either now i know, <u>jaime's</u> <u>a</u> dark sin<u>nner</u>. <s>and</s> The magister bowed slightly. "My lord husband gave her a world" <u>she said</u> to his face that was <s>a</s> grey*

In [16]:
# Попробуем сгенерировать еще несколько текстов
generated_text = generate_text("Lannister", 10, X.shape[1])
print(generated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Lannister mocks us leave us in a rock and the other


In [17]:
generated_text = generate_text("Sister", 10, X.shape[1])
print(generated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Sister we will fight their names in the river and argue


In [13]:
# Сохраняем модель
model.save('GoT_generator.keras')

Статья: https://papers.neurips.cc/paper_files/paper/2016/file/16026d60ff9b54410b3435b403afd226-Paper.pdf

- Чем этот подход отличается от Teacher Forcing?

Teacher Forcing – обучение с учителем (подача эталонных данных на вход). А Professor Forcing – адаптивное состязательное выравнивание динамики.
- Опишите основной принцип предложенного в статье подхода.

Professor Forcing применяет метод адаптации доменов с использованием состязательного обучения, чтобы динамика рекуррентной сети оставалась одинаковой как во время обучения, так и при генерации последовательностей на множестве шагов.
- Какие недостатки есть у подхода Professor Forcing?

Для данного подхода нужно намного больше вычислительных ресурсов. Авторы отмечают, что метод улучшает генерацию аудио, несмотря на <u>короткие</u> обучающие последовательности. Однако это может быть не универсальным: для других задач (например, текста) преимущества могут быть менее выражены.

Статья про LSTM: https://ieeexplore.ieee.org/document/8690387

Статья про GRU: https://arxiv.org/abs/1406.1078

- Опишите основной принцип работы этих моделей

Обе модели используют механизмы вентилей (gates), которые регулируют поток информации, но делают это немного по-разному. LSTM вводит три ключевых вентиля (forget, input, output), а GRU два (reset, update). LSTM поддерживает долговременную память и кратковременную - две переменные. GRU объединяет долговременную и кратковременную память в одно состояние - одна переменная.
Обе модели решают главную проблему RNN — забывание информации на больших дистанциях, но делают это разными способами.

- Что такое гейты и как они помогают избежать типичных проблем RNN?

Гейты — это специализированные механизмы в LSTM и GRU, которые регулируют поток информации с помощью сигмоидных и гиперболических тангенсов функций. Они открываются или закрываются для сохранения полезной информации, забывания ненужной и обновления. Каждый гейт выдаёт значение от 0 (полное блокирование) до 1 (полное пропускание).

У RNN две главные проблемы: исчезающие градиенты, взрывающиеся градиенты и долгосрочная зависимость.
Гейт забывания (Forget Gate в LSTM) контролирует, какие данные сохранить в долговременной памяти. Если информация важна, гейт оставляет её почти нетронутой, предотвращая "затухание" градиентов.
Skip-connections позволяют градиентам течь без затухания через многие временные шаги.
Гейты ограничивают масштаб изменений, и функции активации также держат значения в диапазоне [-1, 1], предотвращая неконтролируемый рост.
Вентиль обновления (Update Gate) в GRU решает, сколько прошлой информации сохранить, а сколько заменить новой, балансируя между кратко- и долгосрочной памятью.

- Насколько надежно использование LSTM / GRU против взрывающегося / исчезающего градиента?

LSTM и GRU не полностью устраняют проблемы градиентов, но делают их менее критичными. LSTM надёжнее для экстремально длинных зависимостей. GRU — хороший компромисс для большинства задач.